In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tab_transformer_pytorch import TabTransformer, FTTransformer
from preprocessing import get_features_and_target
from sklearn.preprocessing import LabelEncoder
from RMSELoss import RMSELoss
import plotly.graph_objects as go

# Getting Dataframe

In [3]:
train_df = pd.read_csv("data/train_data.csv")
dev_df = pd.read_csv("data/development_data.csv")

target_column = "PullTest (N)"  

x_train, y_train = get_features_and_target(train_df, target_column)
x_dev, y_dev = get_features_and_target(dev_df, target_column)



# Encode Categorical_Features

In [4]:
# Define the categorical features
categorical_features = ["Material"]

le = LabelEncoder()

for feature in categorical_features:
    x_train[feature] = le.fit_transform(x_train[feature])
    x_dev[feature] = le.transform(x_dev[feature])  

# Split Categorical_Features

In [5]:
# Drop categorical features to get the continuous features
x_train_numerical_features = x_train.drop(categorical_features, axis=1)
x_dev_numerical_features = x_dev.drop(categorical_features, axis=1)

# Seperate the categorical features
x_train_categorical_features = x_train[categorical_features]
x_dev_categorical_features = x_dev[categorical_features]

# Change df into Tensors

In [156]:
train_tensor = torch.tensor(x_train.to_numpy(), dtype=torch.float)
x_train_numer_tensor = torch.tensor(x_train_numerical_features.to_numpy(),dtype=torch.float)
x_dev_numer_tensor = torch.tensor(x_dev_numerical_features.to_numpy(),dtype=torch.float)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float)

dev_tensor = torch.tensor(x_dev.to_numpy(), dtype=torch.float)
x_train_categorical_features_tensor = torch.tensor(x_train_categorical_features.to_numpy(),dtype=torch.long)
x_dev_categorical_features_tensor = torch.tensor(x_dev_categorical_features.to_numpy(),dtype=torch.long)
y_dev_tensor = torch.tensor(y_dev.to_numpy(), dtype=torch.float)

from torch.utils.data import TensorDataset,DataLoader

train_ds = TensorDataset(
    x_train_categorical_features_tensor,
    x_train_numer_tensor,
    y_train_tensor
)
val_ds = TensorDataset(
    x_dev_categorical_features_tensor,
    x_dev_numer_tensor,
    y_dev_tensor
)
g = torch.Generator()
g.manual_seed(42)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, generator= g)
val_loader   = DataLoader(val_ds,   batch_size=70)



# Check Tensors


In [6]:
#torch.set_printoptions(sci_mode=False, precision=3)
#print(x_train_numer_tensor)

# Define Model

In [170]:
# categories is defined as a tuple, we only have one categorical feature "Material"
# the second parameter has to be empty for the model to work correctly
# in hard numbers this is displaying (1,)
model = FTTransformer(
    categories=(x_train_categorical_features.shape[1],),
    num_continuous=x_train_numerical_features.shape[1],
    dim=9,
    dim_out=1,
    depth=4,
    heads=4,
    attn_dropout=0.1,
    ff_dropout=0.1
)

# (Alternative) Initiate a saved Model

In [ ]:
path = 'trained_models/regular_training/model_FTTransformer_lr0.00025_8_1_4_4_.3_0.3_date_20250810_epoch10000.pt'
model.load_state_dict(torch.load(path))

# Physical Epoch


In [171]:

#["Thickness A (mm)"] = 5
#["Thickness B (mm)"] = 6
# idxA = 5 , idxB = 6

idxA, idxB = 5, 6  

def physics_pull_force(x_cont):
    # x_cont: [B, num_cont_features]
    A = x_cont[:, idxA]               
    B = x_cont[:, idxB]               

    # element‐wise minimum of each row
    t = torch.minimum(A, B)           
    # f_phys = (π/4)*(5*√t)^2*(0.6*365)
    return ((torch.pi / 4) * (4 * torch.sqrt(t)).pow(2) * (0.8 * 365))


def train_one_epoch(train_loader):
    total_error     = 0.0


    for x_cat, x_cont, y in train_loader:
        optimizer.zero_grad()

        # ensure y is [B,1]
        y = y.unsqueeze(-1)
    # [B,1]



        # physics target (no grads needed for f_phys)
        with torch.no_grad():
            f_phys = physics_pull_force(x_cont).unsqueeze(-1)    # [B]
            
        # MSE to the physics‐based target
        True_Error = y - f_phys

        # forward
        pred = model(x_cat, x_cont)   

        # loss
        error_res_loss = criterion(pred, True_Error)

        # combined loss
        loss = error_res_loss
        loss.backward()
        optimizer.step()

        total_error += loss.item()



    return total_error / len(train_loader)




# Physical Error Training

In [ ]:
from datetime import datetime

criterion = RMSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00025)

#Changeable Parameters
# ----------------------------------------------------#
#Model description for saving
model_description    = 'lr0.00025_9_1_4_4_.1_.1Errorbatch70'

# Number of epochs to train
EPOCHS = 10000
#-----------------------------------------------------#

# Lists to store per‐epoch losses
train_phys_losses       = []
val_phys_losses         = []
prediction              = []
physics                 = []  
datasety                = []  


best_vloss   = float('inf')
last_ckpt  = None
#timestamp for manual checkpoint selection
timestamp    = datetime.now().strftime('%Y%m%d_%H%M%S')



for epoch in range(EPOCHS):
    print(f'\nEPOCH {epoch+1}/{EPOCHS}')

    # --------------------
    # 1) TRAINING PHASE
    # --------------------
    model.train()
    train_data = train_one_epoch(train_loader)

    train_phys_losses.append(train_data)
    print(f'train loss: {train_data:.4f}')

    
    # --------------------
    # 2) VALIDATION PHASE
    # --------------------
    model.eval()
    
    val_loss = 0.0
    with torch.no_grad():
        for x_cat, x_cont, y in val_loader:
            y = y.unsqueeze(-1)
            pred = model(x_cat, x_cont)

            f_phys = physics_pull_force(x_cont)    # [B]
            f_phys = f_phys.unsqueeze(-1) 


            True_Error = y - f_phys

            # 1) loss
            val_loss += criterion(f_phys+ pred, f_phys + True_Error).item()

    avg_vloss = val_loss / len(val_loader)


    val_phys_losses.append(avg_vloss)
    prediction.append(f_phys + pred)
    physics.append(f_phys)
    datasety.append(y)


    print(f'valid loss: {avg_vloss:.4f}')
    

    # --------------------
    # 3) CHECKPOINTING
    # --------------------
    # to sort through the checkpoints manually, add timestamp to the filename
    # remove avg_total < best_vloss constraint and 2nd if condition

    #_date_{timestamp}
    if (epoch + 1) % 100 == 0 and avg_vloss < best_vloss:
        if last_ckpt is not None:
            os.remove(last_ckpt)

        best_vloss = avg_vloss
        ckpt_path = f'trained_models/physical_error_training/model_{model_description}_long_epoch{epoch+1}.pt'
        torch.save(model.state_dict(), ckpt_path)
        last_ckpt = ckpt_path



EPOCH 1/10000
train loss: 705.2603
valid loss: 745.8729

EPOCH 2/10000
train loss: 706.0775
valid loss: 745.7097

EPOCH 3/10000
train loss: 704.3992
valid loss: 745.5555

EPOCH 4/10000
train loss: 707.8783
valid loss: 745.4268

EPOCH 5/10000
train loss: 711.9343
valid loss: 745.3392

EPOCH 6/10000
train loss: 700.2340
valid loss: 745.3163

EPOCH 7/10000
train loss: 700.2549
valid loss: 745.2933

EPOCH 8/10000
train loss: 703.8455
valid loss: 745.2721

EPOCH 9/10000
train loss: 700.0929
valid loss: 745.2528

EPOCH 10/10000
train loss: 710.7512
valid loss: 745.2356

EPOCH 11/10000
train loss: 702.1705
valid loss: 745.2200

EPOCH 12/10000
train loss: 705.2988
valid loss: 745.2054

EPOCH 13/10000
train loss: 704.4826
valid loss: 745.1919

EPOCH 14/10000
train loss: 701.6953
valid loss: 745.1791

EPOCH 15/10000
train loss: 706.9523
valid loss: 745.1667

EPOCH 16/10000
train loss: 699.5437
valid loss: 745.1548

EPOCH 17/10000
train loss: 704.4277
valid loss: 745.1429

EPOCH 18/10000
train l

# Plotly

In [174]:
import plotly.graph_objects as go

epochs = list(range(1, len(train_phys_losses) + 1))

fig = go.Figure()

# Training loss trace
fig.add_trace(go.Scatter(
    x=epochs,
    y=train_phys_losses,
    mode="lines",
    name="Train Loss",
    line=dict(color="blue", width=2)
))

# Validation loss trace
fig.add_trace(go.Scatter(
    x=epochs,
    y=val_phys_losses,
    mode="lines",
    name="Validation Loss",
    line=dict(color="firebrick", width=2, dash="dash")
))

# Layout enhancements
fig.update_layout(
    title="Training & Validation Loss over 10 000 Epochs - lr0.0025_9_1_4_4_.1_.1 - Loss on Physical Error",
    xaxis_title="Epoch",
    yaxis_title="Loss",
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(0, 50500, 2500)),  # Show ticks every 100 epochs
        tickfont=dict(size=10)
    ),
    yaxis=dict(
        tickformat=".2e" if max(train_phys_losses + val_phys_losses) > 1e4 else ".4f",  # Dynamic formatting
        gridcolor="lightgray"
    ),
    legend=dict(
        x=0.7,        # push legend past the right edge
        y=1,
        xanchor="left",
        borderwidth=1
    ),
    
    template="plotly_white",
    margin=dict(t=60, b=40)
)

fig.show()


In [187]:
# After training, do a one‐pass over val_loader:
all_true, all_phys, all_pred = [], [], []

with torch.no_grad():
    for x_cat, x_cont, y in val_loader:
        y   = y.unsqueeze(-1)                             # [B,1]
        f_p = physics_pull_force(x_cont).unsqueeze(-1)     # [B,1]
        r_p = model(x_cat, x_cont)                         # [B,1]
        y_p = (f_p + r_p)                                  # [B,1]

        all_true.append(y.ravel())
        all_phys.append(f_p.ravel())
        all_pred.append(y_p.ravel())

# flatten
all_true = np.concatenate(all_true)
all_phys = np.concatenate(all_phys)
all_pred = np.concatenate(all_pred)

# now plot per‐sample
sample_idx = np.arange(len(all_true))

fig = go.Figure()
fig.add_trace(go.Scatter(x=sample_idx, y=all_true, mode="lines+markers",
                         name="True Data", marker=dict(color="black", size=6)))
fig.add_trace(go.Scatter(x=sample_idx, y=all_phys, mode="lines+markers",
                         name="Physics Only", marker=dict(color="firebrick", size=6)))
fig.add_trace(go.Scatter(x=sample_idx, y=all_pred, mode="lines+markers",
                         name="Prediction", marker=dict(color="blue", size=6)))

fig.update_layout(
    title="All Validation Samples: True vs Physics vs Prediction Epochs 10 000 lr0.0025_9_1_4_4_.1_.1",
    xaxis_title="Sample Index",
    yaxis_title="Pull-Force",
    template="seaborn"
)
fig.show()


In [ ]:
# Collect predictions and true values from the validation set
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for x_cat, x_cont, y in val_loader:
        y = y.unsqueeze(-1)               
        all_targets.append(y)

        # 1) Physics pull‐force 
        F_phys = physics_pull_force(x_cont).unsqueeze(-1)

        # 2) Error
        True_Error = model(x_cat, x_cont)

        # 3) full prediction
        F_pred = (F_phys + True_Error)
        all_preds.append(F_pred)

# Concatenate batches
y_true = np.vstack(all_targets)
y_pred = np.vstack(all_preds)

# Calculate MAE and RMSE and R2
mae  = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
R2   = r2_score(y_true, y_pred)

print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2: {R2:.2f}")

MAE:  186.19
RMSE: 289.07
R2: 0.63


# Plotly(old representation)

In [18]:
import plotly.graph_objects as go

fig = go.Figure()

# Training loss trace
fig.add_trace(go.Scatter(
    y=train_data_losses,
    mode="lines+markers",
    name="Train Data Loss",
    line=dict(color="lightblue", width=2),
    marker=dict(size=4)
))

# Validation loss trace
fig.add_trace(go.Scatter(
    y=val_data_losses,
    mode="lines+markers",
    name="Validation Data Loss",
    line=dict(color="orange", width=2),
    marker=dict(size=4)
))

# Layout
fig.update_layout(
    title="Training & Validation Loss over Epochs  - lr0.00025_8_1_4_4_.3_.3 - Physics Lambda 4",
    xaxis_title="Epoch",
    yaxis_title="Loss",
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(0, 10500, 500)),  # Show ticks every 100 epochs
        tickfont=dict(size=10)
    ),
    template="plotly_white",
    legend=dict(
        x=0.7,        # push legend past the right edge
        y=1,
        xanchor="left",
        borderwidth=1
    ),
    margin=dict(t=60, b=40)  # give extra room on right for the legend
)

fig.show()
